In [ ]:
!apt-get update
!apt-get install -y poppler-utils
! pip install pdf2image byaldi qwen-vl-utils bitsandbytes

In [ ]:
import os
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

# 任意のPDFパス
pdf_path = "sample.pdf"

# ファイルがない場合の処理
if not os.path.exists(pdf_path):
    print(f"{pdf_path}が指定されたPDFファイルが存在しません。")
else:
    print(f"PDFを読み込んでいます: {pdf_path}")

    # PDFの1ページ目を画像に変換して表示
    images = convert_from_path(pdf_path)
    sample_image = images[0]

    plt.figure(figsize=(10, 10))
    plt.imshow(sample_image)
    plt.axis('off')
    plt.title("Target Document Page 1")
    plt.show()
    print(f"全ページ：{len(images)}ページ")

In [ ]:
from byaldi import RAGMultiModalModel

# Load a Model
RAG = RAGMultiModalModel.from_pretrained("vidore/colpali-v1.2", verbose=0)

# インデックスの作成
index_name = "my_pdf_index"

print("Indexing start...")

RAG.index(
    input_path=pdf_path,
    index_name=index_name,
    store_collection_with_index=True,
    overwrite=True
)
print("Indexing done.")

In [ ]:
# 検索クエリ
query_text = "このドキュメントに記されている図は何を示していますか？"

# 検索実行
results = RAG.search(query_text, k=1)

# 検索結果
print(f"Query: {query_text}")
print(f"Top Result Score: {results[0].score:.4f}")
print(f"Retrieved Page ID: {results[0].doc_id}, Page Index: {results[0].page_num}")

# ヒットしたページの画像を取得
page_index = results[0].page_num - 1
retrieved_image_content = images[page_index]

# 検索されたページを可視化
plt.figure(figsize=(10, 10))
plt.imshow(retrieved_image_content)
plt.axis('off')
plt.title(f"Page for Query: {query_text}")
plt.show()

In [ ]:
import gc
import torch

# メモリの掃除

# Retrieverモデルをメモリから削除
del RAG
del results
gc.collect()
torch.cuda.empty_cache()

print("VRAM Cleaned!")

In [ ]:
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info

# メモリ不足対策：4bit量子化
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Generator model
gen_model_name = "Qwen/Qwen2-VL-7B-Instruct"

print(f"Loading generator: {gen_model_name}")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    gen_model_name,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# Processor
processor = AutoProcessor.from_pretrained(gen_model_name)

# 検索された画像を入力として与える
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": retrieved_image_content,
            },
            {"type": "text", "text":f"与えられた画像を元に、次の質問を詳しく答えてください。: {query_text}"},
        ],
    }
]

In [ ]:
# 推論の準備
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to("cuda")

# 生成実行
print("Generating...")
generated_ids = model.generate(**inputs, max_new_tokens=512)
generate_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generate_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print("-" * 30)
print("[AI Answer]")
print(output_text[0])
print("-" * 30)